In [1]:
import numpy as np
import matplotlib.pyplot as plt
from MDAnalysis import Universe, Writer
from MDAnalysis.analysis import align
import pickle
from tqdm.auto import tqdm, trange

/home/wasim/miniconda3/lib/python3.11/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


# decomposition

In [2]:
traj = Universe("data/dopc/dopc.pdb", "data/dopc/dopc.xtc")
ref = Universe("data/dopc/dopc.pdb").select_atoms("all")
select = traj.select_atoms("all")

Nlipids = len(set(ref.resids))
block_size = select.positions.shape[0]//Nlipids

ref_pos = ref.select_atoms("resid 1").positions

/home/wasim/miniconda3/lib/python3.11/site-packages/MDAnalysis/coordinates/XDR.py:240: UserWarning: Reload offsets from trajectory
 ctime or size or n_atoms did not match
  warnings.warn("Reload offsets from trajectory\n "


In [3]:
# Select all lipids positions directly and convert to a NumPy array
pos0 = np.array([traj.select_atoms("all").positions for t in tqdm(traj.trajectory)])

# Reshape the array to get decomposed positions
pos_decomposed = pos0.reshape(-1, pos0.shape[1] // Nlipids, 3)

# # Compute center of geometry (cog) more efficiently
cog = pos_decomposed.mean(axis=1, keepdims=True)

# # Vectorized computation of rotation matrices
ref_cog = ref_pos.mean(axis=0)
ref_aligned = ref_pos - ref_cog

# # Vectorized broadcasting of cog subtraction
pos_centered = pos_decomposed - cog

# # Vectorized computation of rotation matrices using np.einsum
R = np.array([align.rotation_matrix(pc, ref_aligned)[0] for pc in tqdm(pos_centered)])

# # Apply the rotation matrices to the positions
pos_processed = np.einsum('nij,nkj->nki', R, pos_centered)

  0%|          | 0/3159 [00:00<?, ?it/s]

  0%|          | 0/581256 [00:00<?, ?it/s]

with Writer("data/dopc/single.pdb") as w:
    _ = ref.select_atoms("resid 1")
    _.positions = ref_pos - ref_pos.mean(axis=0)
    w.write(_)

with Writer("data/dopc/single.xtc") as w:
    _ = ref.select_atoms("resid 1")
    for p in pos_processed:
        _.positions = p
        w.write(_)

# reconstruct membrane

In [28]:
ref = Universe("data/dopc/dopc.pdb").select_atoms("all")

select = ref.select_atoms("all")

Nlipids = len(set(select.resids))

In [28]:
pos_reverted = np.einsum('nij,nkj->nki', R.transpose(0, 2, 1), pos_processed) + cog